In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import os
import copy

# Factorization Machine

In [2]:
users_movies = ['Alice', 'Bob', 'Chris',
                            'TITANIC', 'NOTTING HILL', 'STARWARS', 'STARTREK']
data = pd.DataFrame([
    [1,0,0,1,0,0,0],
    [1,0,0,0,1,0,0],
    [1,0,0,0,0,1,0],
    [0,1,0,0,0,1,0],
    [0,1,0,0,0,0,1],
    [0,0,1,1,0,0,0],
    [0,0,1,0,0,1,0]], 
    columns=users_movies)

target = [5,3,1,4,5,1,5]

print(target)
data

[5, 3, 1, 4, 5, 1, 5]


,Alice,Bob,Chris,TITANIC,NOTTING HILL,STARWARS,STARTREK
0,1,0,0,1,0,0,0
1,1,0,0,0,1,0,0
2,1,0,0,0,0,1,0
3,0,1,0,0,0,1,0
4,0,1,0,0,0,0,1
5,0,0,1,1,0,0,0
6,0,0,1,0,0,1,0


## 原理 

次の式をNNで再現する．  
$$ \hat{y}({\bf x}) = w_0 + \sum^N_{i=1} w_i x_i + \sum^N_{i=1} \sum^N_{j=i+1} (v_i \cdot v_j) x_i x_j $$
第1項，第2項は線形モデル，つまりNNの全結合層で表せる．  
第3項は次のようになる．  
$$ \sum^N_{i=1} \sum^N_{j=i+1} (v_i \cdot v_j) x_i x_j = \frac{1}{2} \sum_{f=1}^{k} \Big( \big(\sum_{i=1}^{n} v_f^{(i)} x_i \big)^2 - \sum_{i=1}^{n}v_f^{(i) 2} x_i^2 \Big) \\ = 
\frac{1}{2} \sum_{f=1}^{} \Big( S_{1,f}^2 - S_{2,f} \Big)\\ =
\frac{1}{2} \Big( S_{1}^2 - S_{2} \Big) $$

データ数$N$，特徴次元数$d$，分解次元数(ハイパーパラメータ)$k$とおくと，
x.shape = $(N, d) $  
v.shape = $(d, k) $

$XV$を計算すると，次のような要素が現れるので，要素ごとに2乗して合計をとれば$S_1^2$が求まる．  
同様に，$X$と$V$を要素ごとに2乗して，それらを掛けて合計を取れば$S_2$が求まる．($vx$の積で表現した第3項と対応する要素が$XV$行列にある)

$$
XV = \begin{bmatrix}
\sum_{i=1}^{n} v_f^{(1)} x_i^{(1)}  & \dots &  \sum_{i=1}^{n} v_f^{(k)} x_i^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\ 
\sum_{i=1}^{n} v_f^{(1)} x_i^{(M)} & \dots & \sum_{i=1}^{n} v_f^{(k)} x_i^{(M)} \\
\end{bmatrix} = 
\begin{bmatrix}
S_{11}^{(1)}  & \dots &  S_{1k}^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\ 
S_{11}^{(M)}  & \dots & S_{1k}^{(M)} \\
\end{bmatrix}$$

したがって，最終的に実装する式は次のようになる．  
$$\hat{y}({\bf X}) = w_0 + \sum^N_{i=1} w_i x_i + \frac{1}{2} \sum_{row}( f(XV) - f(X) \times f(V))$$
$$f : {\rm element \, wise \, square}$$

In [3]:
class FactorizationMachine(nn.Module):
    def __init__(self, dim=None, k=None):
        super().__init__()
        self.V = nn.Parameter(torch.randn(dim,k), requires_grad=True)
        self.fc = nn.Linear(dim, 1)
        
    def forward(self, x):
        linear_term = self.fc(x)
        s1_squared = torch.matmul(x, self.V)\
                                          .pow(2)\
                                          .sum(1, keepdim=True)
        s2 = torch.matmul(x.pow(2), self.V.pow(2))\
                          .sum(1, keepdim=True)
        out = .5 * (s1_squared - s2)
        out = out + linear_term
        return out
    
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
# main training function
def train_mlp(X, X_test, y, folds, model_class=None, model_params=None, batch_size=128, epochs=1,
              criterion=None, optimizer_class=None, opt_params=None,
#               clr=cyclical_lr(10000),
              device=None):
    
    seed_everything()
    models = []
    scores = []
    train_preds = np.zeros(y.shape)
    test_preds = np.zeros((X_test.shape[0], 1))
    
    X_tensor, X_test, y_tensor = torch.from_numpy(X).to(device), torch.from_numpy(X_test).to(device), torch.from_numpy(y).to(device)
    for n_fold, (train_ind, valid_ind) in enumerate(folds.split(X, y)):
        
        print(f'fold {n_fold+1}')
        
        train_set = TensorDataset(X_tensor[train_ind], y_tensor[train_ind])
        valid_set = TensorDataset(X_tensor[valid_ind], y_tensor[valid_ind])
        
        loaders = {'train': DataLoader(train_set, batch_size=batch_size, shuffle=True),
                   'valid': DataLoader(valid_set, batch_size=batch_size, shuffle=False)}
        
        model = model_class(**model_params)
        model.to(device)
        best_model_wts = copy.deepcopy(model.state_dict())
        
        optimizer = optimizer_class(model.parameters(), **opt_params)
        
        # training cycle
        best_score = 0.
        for epoch in range(epochs):
            losses = {'train': 0., 'valid': 0}
            
            for phase in ['train', 'valid']:
               
                if phase == 'train':
                    model.train()
                else:
                    model.eval()
                
                for batch_x, batch_y in loaders[phase]:
                    optimizer.zero_grad()
                    out = model(batch_x)
                    loss = criterion(out, batch_y)
                    losses[phase] += loss.item()*batch_x.size(0)
                    
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                losses[phase] /= len(loaders[phase].dataset)
            
            # after each epoch check if we improved roc auc and if yes - save model
            with torch.no_grad():
                model.eval()
                valid_preds = sigmoid(model(X_tensor[valid_ind]).cpu().numpy())
                try:
                    epoch_score = roc_auc_score(y[valid_ind], valid_preds)
                except:
                    epoch_score = 0.5
                if epoch_score > best_score:
                    best_model_wts = copy.deepcopy(model.state_dict())
                    best_score = epoch_score
            
            if ((epoch+1) % 30) == 0:
                print(f'ep {epoch+1} loss: {losses["train"]:.3f} loss_val {losses["valid"]:.3f} auc_val {epoch_score:.3f}')
        
        # prediction on valid set
        with torch.no_grad():
            model.load_state_dict(best_model_wts)
            model.eval()
            
            train_preds[valid_ind] = sigmoid(model(X_tensor[valid_ind]).cpu().numpy())
            try:
                fold_score = roc_auc_score(y[valid_ind], train_preds[valid_ind])
            except:
                fold_score = 0.5
            scores.append(fold_score)
            print(f'Best ROC AUC score {fold_score}')
            models.append(model)

            test_preds += sigmoid(model(X_test).cpu().numpy())
    
    print('CV AUC ROC', np.mean(scores), np.std(scores))
    
    test_preds /= folds.n_splits
    
    return models, train_preds, test_preds    

In [5]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [6]:
train_df = data
test_df = pd.DataFrame([[1,0,0,0,0,0,1], [0,1,0,1,0,0,0]], columns=users_movies)

test_df

,Alice,Bob,Chris,TITANIC,NOTTING HILL,STARWARS,STARTREK
0,1,0,0,0,0,0,1
1,0,1,0,1,0,0,0


In [7]:
X_train = train_df.values.astype(np.float32)
X_test = test_df.values.astype(np.float32)
y = np.array(target).astype(np.float32).reshape(-1,1)

In [8]:
MS, train_preds, test_preds = train_mlp(X_train, X_test, y,
                            KFold(n_splits=5, random_state=17), 
                            model_class=FactorizationMachine, 
                            model_params={'dim': X_train.shape[1], 'k': 5}, 
                            batch_size=2,
                            epochs=50,
                            criterion=nn.BCEWithLogitsLoss(),
                            optimizer_class=torch.optim.SGD, 
                            opt_params={'lr': 0.01, 'momentum': 0.9},
                            device=DEVICE
                            )

fold 1


/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


ep 30 loss: -142172.350 loss_val -56.231 auc_val 0.500
Best ROC AUC score 1.0
fold 2
ep 30 loss: -50096.438 loss_val -25323.180 auc_val 0.500
Best ROC AUC score 1.0
fold 3
ep 30 loss: -15703.262 loss_val -134.576 auc_val 0.500
Best ROC AUC score 0.5
fold 4
ep 30 loss: -144916.516 loss_val 0.000 auc_val 0.500
Best ROC AUC score 0.5
fold 5
ep 30 loss: -24950.921 loss_val -5.176 auc_val 0.500
Best ROC AUC score 0.5
CV AUC ROC 0.7 0.2449489742783178


/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/chieda/anaconda3/envs/torchenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/chieda/anaconda3/envs/torchenv/lib/python

In [9]:
train_preds

array([[0.72969955],
       [0.47967553],
       [0.25821853],
       [0.97765607],
       [0.15790828],
       [0.9304136 ],
       [0.82147592]])

In [10]:
test_preds

array([[0.5161366 ],
       [0.29246324]])

In [11]:
target

[5, 3, 1, 4, 5, 1, 5]